# Example ANN generation and training

From melon doc (https://git.rwth-aachen.de/avt-svt/public/MeLOn/-/blob/master/feedforward%20neural%20network/training/keras/example_training_of_ANN.py) and the ANN paper (https://link.springer.com/content/pdf/10.1007/s10957-018-1396-0.pdf).

In [1]:
import utils
import tensorflow as tf
import numpy as np
import time
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from pyDOE import lhs
import maingopy

Implement peaks function

## Optimize with maingopy

In [2]:
# To define a model, we need to spcecialize the MAiNGOmodel class
class Model(maingopy.MAiNGOmodel):
    def __init__(self):
        maingopy.MAiNGOmodel.__init__(self)
        # Initialize feedforward neural network and load data from example csv file
        self.ffANN = maingopy.melonpy.FeedForwardNet()
        # folder where the model xml is stored
        self.path = "./data"
        # xml filename
        self.fileName= "myTestANN"
        # open them (define that it is an XML instead of a CSV)
        self.ffANN.load_model(self.path, self.fileName, maingopy.melonpy.CSV)

    # We need to implement the get_variables functions for specifying the optimization varibles
    def get_variables(self):
        # define bounds of the original variables, so that it rescales the results fo the optimization
        # the optimization is done with the normalized version of these values
        variables = [ maingopy.OptimizationVariable(maingopy.Bounds(-3,3), maingopy.VT_CONTINUOUS, "x"),
                      maingopy.OptimizationVariable(maingopy.Bounds(-3,3), maingopy.VT_CONTINUOUS, "y") ]
        return variables

    # We need to implement the evaluate function that computes the values of the objective and constraints from the variables.
    # Note that the variables in the 'vars' argument of this function do correspond to the optimization variables defined in the get_variables function.
    # However, they are different objects for technical reasons. The only mapping we have between them is the position in the list.
    # The results of the evaluation (i.e., objective and constraint values) need to be return in an EvaluationContainer
    def evaluate(self, vars):
        x = vars[0]
        y = vars[1]
        
        # Inputs to the ANN are the variables x and y
        annInputs = [x, y]
        
        # Evaluate the network (in reduced-space)
        # This returns a list, because the output of the network may be multidimensional
        annOutputs = self.ffANN.calculate_prediction_reduced_space(annInputs)

        # Set the ANN output (only 1 in this case) as objective to be minimized
        result = maingopy.EvaluationContainer()
        result.objective = annOutputs[0]

        return result

In [3]:
# To work with the problem, we first create an instance of the model.
myModel = Model()
# We then create an instance of MAiNGO, the solver, and hand it the model.
myMAiNGO = maingopy.MAiNGO(myModel)

In [4]:
# We can have MAiNGO read a settings file:
fileName = ""
myMAiNGO.read_settings(fileName) # If fileName is empty, MAiNGO will attempt to open MAiNGOSettings.txt
myMAiNGO.set_log_file_name(".logs/my_log_file.log")
myMAiNGO.set_option("writeCsv", True)
myMAiNGO.set_iterations_csv_file_name(".logs/iterations.csv")
myMAiNGO.set_solution_and_statistics_csv_file_name(".logs/solution_and_statistics.csv")

In [5]:
myMAiNGO.write_model_to_file_in_other_language(writingLanguage=maingopy.LANG_GAMS, fileName="./logs/my_problem_file_MAiNGO.gms", solverName="SCIP", writeRelaxationOnly=False)

In [6]:
# Finally, we call the solve routine to solve the problem.
maingoStatus = myMAiNGO.solve()
print(maingoStatus)

RETCODE.GLOBALLY_OPTIMAL


In [7]:
# dir(myMAiNGO)

In [8]:
myMAiNGO.get_solution_point()

[0.22761935531019054, -1.6239431446898094]

In [9]:
myMAiNGO.get_objective_value()

-6.552977624048905

In [10]:
myMAiNGO.get_status()

<RETCODE.GLOBALLY_OPTIMAL: 0>

In [11]:
myMAiNGO.get_cpu_solution_time()

1.125

In [12]:
myMAiNGO.evaluate_model_at_point(np.array([0.228, -1.626]))

([-6.5530061614914015], True)